In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    X = data.drop(columns=['Eligible'])
    y = data['Eligible']
    return X, y

def optimize_knn_model(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    knn_model = KNeighborsRegressor()
    param_grid = {
        'n_neighbors': [3, 5, 10, 20, 30, 40],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan'],
        'p': [1, 2]  # 1: Manhattan distance, 2: Euclidean distance
    }
    grid_search = GridSearchCV(knn_model, param_grid, cv=5, verbose=1, scoring='neg_mean_squared_error')
    grid_search.fit(X_train_scaled, y_train)
    print("Best parameters:", grid_search.best_params_)
    best_knn = grid_search.best_estimator_
    return best_knn, scaler

# Load and train on VehicleTrainingDataset.csv
X_train, y_train = load_and_preprocess_data('VehicleTrainingDataset_Noisy_0.01.csv')
knn_model, scaler = optimize_knn_model(X_train, y_train)

# Predict eligibility scores on 1000VehicleDataset.csv
vehicles_df = pd.read_csv('1000VehicleDataset_Noisy_0.01.csv')
X_test = vehicles_df.drop(columns=['Eligible'])
X_test_scaled = scaler.transform(X_test)
predicted_scores = knn_model.predict(X_test_scaled)

# Assuming you have access to actual scores, replace this line with the actual score loading logic if available
y_actual = vehicles_df['Eligible']

# Replace actual scores with predicted ones
vehicles_df['Eligible'] = predicted_scores  

# Calculate metrics
mae = mean_absolute_error(y_actual, predicted_scores)
rmse = np.sqrt(mean_squared_error(y_actual, predicted_scores))
r_squared = r2_score(y_actual, predicted_scores)
rae = np.sum(np.abs(y_actual - predicted_scores)) / np.sum(np.abs(y_actual - np.mean(y_actual)))

# Output the results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")
print(f"RAE: {rae}")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'metric': 'manhattan', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
MAE: 1.7258836094021666
RMSE: 2.159748576425794
R-squared: 0.9684512036561819
RAE: 0.18025169439477964


In [2]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks400.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-------- Rollout Summary --------
Total mean reward: -386.0
Standard deviation of reward: 0.0
Average successful assignments: 8.083333333333334
All assignments history: [22, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 400      |
|    ep_rew_mean     | -373     |
| time/              |          |
|    fps             | 117      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -288.0
Standard deviation of reward: 0.0
Average successful assignments: 28.458333333333332
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 400          |
|    ep_rew_mean          | -374         |
| time/                   |              |
|    fps                  | 103          |
|    iterations           | 2            |
|    time_elapsed         | 19           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0076119294 |
|    clip_fraction        | 0.0771       |
|    clip_range           | 0.15         |
|    entropy_loss         | -5.87        |
|    explained_variance   | 0.0534       |
|    learning_rate        | 0.00018      |
|    loss                 | 3.63         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0403      |
|    value

-------- Rollout Summary --------
Total mean reward: -10.0
Standard deviation of reward: 0.0
Average successful assignments: 76.55833333333334
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 400         |
|    ep_rew_mean          | -367        |
| time/                   |             |
|    fps                  | 114         |
|    iterations           | 10          |
|    time_elapsed         | 89          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.010334663 |
|    clip_fraction        | 0.195       |
|    clip_range           | 0.15        |
|    entropy_loss         | -5.81       |
|    explained_variance   | 0.00272     |
|    learning_rate        | 0.00018     |
|    loss                 | 1.3         |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.0432     |
|    value_loss           | 4.3

...

-------- Rollout Summary --------
Total mean reward: 168.0
Standard deviation of reward: 0.0
Average successful assignments: 209.6161616161616
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 400         |
|    ep_rew_mean          | -170        |
| time/                   |             |
|    fps                  | 145         |
|    iterations           | 66          |
|    time_elapsed         | 462         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.005836803 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.15        |
|    entropy_loss         | -3.33       |
|    explained_variance   | 0.44        |
|    learning_rate        | 0.00018     |
|    loss                 | 1.28        |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.0357     |
|    value_loss           | 3.3

-------- Rollout Summary --------
Total mean reward: 170.0
Standard deviation of reward: 0.0
Average successful assignments: 215.99324324324326
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 400         |
|    ep_rew_mean          | -119        |
| time/                   |             |
|    fps                  | 153         |
|    iterations           | 74          |
|    time_elapsed         | 495         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.006310312 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.15        |
|    entropy_loss         | -3.15       |
|    explained_variance   | 0.455       |
|    learning_rate        | 0.00018     |
|    loss                 | 1.48        |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0358     |
|    value_loss           | 3.

-------- Rollout Summary --------
Total mean reward: 174.0
Standard deviation of reward: 0.0
Average successful assignments: 221.34552845528455
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 400          |
|    ep_rew_mean          | -75.7        |
| time/                   |              |
|    fps                  | 159          |
|    iterations           | 82           |
|    time_elapsed         | 527          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0064551607 |
|    clip_fraction        | 0.128        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.94        |
|    explained_variance   | 0.401        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.34         |
|    n_updates            | 810          |
|    policy_gradient_loss | -0.0372      |
|    value_

-------- Rollout Summary --------
Total mean reward: 180.0
Standard deviation of reward: 0.0
Average successful assignments: 226.1212962962963
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 400         |
|    ep_rew_mean          | -41.3       |
| time/                   |             |
|    fps                  | 165         |
|    iterations           | 90          |
|    time_elapsed         | 558         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.006051955 |
|    clip_fraction        | 0.118       |
|    clip_range           | 0.15        |
|    entropy_loss         | -2.93       |
|    explained_variance   | 0.501       |
|    learning_rate        | 0.00018     |
|    loss                 | 1.59        |
|    n_updates            | 890         |
|    policy_gradient_loss | -0.034      |
|    value_loss           | 3.4

-------- Rollout Summary --------
Total mean reward: 180.0
Standard deviation of reward: 0.0
Average successful assignments: 230.22619047619048
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 400          |
|    ep_rew_mean          | -18.9        |
| time/                   |              |
|    fps                  | 171          |
|    iterations           | 98           |
|    time_elapsed         | 586          |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0069248946 |
|    clip_fraction        | 0.141        |
|    clip_range           | 0.15         |
|    entropy_loss         | -2.91        |
|    explained_variance   | 0.41         |
|    learning_rate        | 0.00018      |
|    loss                 | 1.24         |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.0366      |
|    value_

In [4]:
# Load new task dataset for testing
new_tasks_df = pd.read_csv('RandomTasks400Test.csv')
new_tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)
test_env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, new_tasks_df), n_envs=1)

# Evaluate the model on the new test environment
mean_reward, std_reward = evaluate_policy(model, test_env, n_eval_episodes=10)
# Extract the successful assignments history from the test environment
successful_assignments = test_env.envs[0].env.get_average_success()

print("---- Testing Summary ----")
print(f"Mean Reward: {mean_reward}")
print(f"Standard Deviation of Reward: {std_reward}")
print(f"Average Successful Assignments in Testing: {successful_assignments}")

C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---- Testing Summary ----
Mean Reward: 172.0
Standard Deviation of Reward: 0.0
Average Successful Assignments in Testing: 286.0
